# Data Augmentation of MNIST Data

Augment existing training data by shifting MNIST image in directions (left, right, up or down) by one pixel for each image.
Train the best model (KNN) on the expanded dataset and measure its accuracy on the test set

First, load the MNIST dataset

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784", version=1)

# view the keys
mnist.keys()

Get X and y

In [ ]:
X, y = mnist['data'], mnist['target']

Split data into train and test sets

In [ ]:
X_train, y_train, X_test, y_test = X[:60000], y[:60000], X[60000:], y[60000:]

View image from the data

In [ ]:
some_digit = X_train[0]
some_digit_image = some_digit.reshape(28, 28)

import matplotlib.pyplot as plt
plt.imshow(some_digit_image, cmap='binary')

View the corresponding label (y)

In [ ]:
y[0]

Now, we want to add / augment more data to the training set by moving the image by 1 pixel in all directions. Let's define
a function which takes in an image and moves it by 1 pixel in a specified direction

In [ ]:
import scipy.ndimage as img
def shift_image(data, direction='left', by_pixels=1):
    """
    Shifts the input image in a specific direction by a the specific number of pixels
    :param data: 1D image data
    :param direction: direction in which to shift the image. Values are ['left', 'right', 'up', 'down']
    :param by_pixels: number of pixels (dint) to shift the image by
    :return: shifted image 2D array
    """
    image = data.reshape(28,28)
    move_image_by = None
    if direction == 'left':
        move_image_by = (0, -by_pixels)
    elif direction == 'right':
        move_image_by = (0, by_pixels)
    elif direction == 'up':
        move_image_by = (-by_pixels, 0)
    elif direction == 'down':
        move_image_by = (by_pixels, 0)

    return img.shift(image, move_image_by).reshape([-1])

new_image = shift_image(some_digit, direction='down', by_pixels=5)
plt.imshow(new_image.reshape(28, 28), cmap='binary')

Shift X_train data by 1 px in all directions

In [ ]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

for direction in ['left', 'right', 'up', 'down']:
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, direction))
        y_train_augmented.append(label)

X_train_augmented

In [ ]:
import numpy as np
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)
X_train_augmented.shape


Shuffle the data

In [ ]:
shuffled_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffled_idx]
y_train_augmented = y_train_augmented[shuffled_idx]

In [ ]:
X_train_augmented.shape, y_train_augmented.shape

Train the model using KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=4, weights='distance') # use the best parameters from the previously evaluated model
knn_clf.fit(X_train_augmented, y_train_augmented)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = knn_clf.predict(X_test)


In [ ]:
accuracy_score(y_test, y_pred)

